In [ ]:
#一次处理多个文档
#configurations 只包含chain完全在pore里

import pandas as pd


In [ ]:
#1_rearrange
#1_add chain number, time step

#load file
#EPS = ['0.20', '0.40', '0.60', '0.80', '1.00', '1.20', '1.40', '1.60', '1.80', '2.00']
#REP = ['01', '02', '03']

EPS = ['2.00']
REP = ['01']

for z in EPS:
    for i in REP:
        fread1 = open('D:\\all\\projects\\python_research_code\\py_count_chains_in_pore\\20221119_NA120_PHIA0.05_NB40_PHIB0.05_R3_EPS' + z + '_REP' + i + '\\long.lammpstrj', 'r')
        longread1 = fread1.readlines()
        fread2 = open('D:\\all\\projects\\python_research_code\\py_count_chains_in_pore\\20221119_NA120_PHIA0.05_NB40_PHIB0.05_R3_EPS' + z + '_REP' + i + '\\short.lammpstrj', 'r')
        shortread1 = fread2.readlines()
        
        # strip, split and save data
        longread2 = []
        shortread2 = []
        for j in longread1:
            first = j.strip('/n')
            second = first.split()
            longread2.append(second)
        for j in shortread1:
            first = j.strip('/n')
            second = first.split()
            shortread2.append(second)
            
        #delete title in dump file
        #8 means the length of the line that should be used for coding in dump file
        longdump = []
        shortdump = []
        for j in range(0, len(longread2), 1):
            if len(longread2[j]) == 8:
                longdump.append(longread2[j])
        for j in range(0, len(shortread2), 1):
            if len(shortread2[j]) == 8:
                shortdump.append(shortread2[j])
                
        # add time step
        timesteplong = []
        timestepshort = []
        idumpstep = 100100000 # timestep the dump start
        dumpsegment = 100000  # dump every time step
        totalparticleslong = 6240  # atoms number
        totalparticlesshort = 6240  # atoms number
        for j in range(1, len(longdump) // totalparticleslong + 1, 1):
            for k in range(1, totalparticleslong + 1, 1):
                timesteplong.append(idumpstep)
            idumpstep = idumpstep + dumpsegment
        idumpstep = 100100000  # reset the initial timestep
        for j in range(1, len(shortdump) // totalparticlesshort + 1, 1):
            for k in range(1, totalparticlesshort + 1, 1):
                timestepshort.append(idumpstep)
            idumpstep = idumpstep + dumpsegment
            
        #add chain number
        longchainnumber = []
        shortchainnumber = []
        longchainlength = 120 #chain length
        shortchainlength = 40 #chain length
        totalparticleslong = 6240 #atoms number
        totalparticlesshort = 6240  #atoms number
        for j in range(1, len(longdump)//totalparticleslong+1, 1):
            p = 0
            for k in range(1, totalparticleslong//longchainlength+1, 1):
                p = p + 1
                for h in range(1, longchainlength+1, 1):
                    longchainnumber.append(p)
        for j in range(1, len(shortdump)//totalparticlesshort+1, 1):
            p = 0
            for k in range(1, totalparticlesshort//shortchainlength+1, 1):
                p = p + 1
                for h in range(1, shortchainlength+1, 1):
                    shortchainnumber.append(p)
                    
        #write dump into a new file
        fwrite1 = open('long_rearrange.txt', 'w')
        fwrite2 = open('short_rearrange.txt', 'w')
        a = 'ID TYPE x y z ix iy iz ' + '\n'
        d = 'ID TYPE x y z ix iy iz ' + '\n'
        fwrite1.write(a)
        fwrite2.write(d)
        for j in range(0, len(longchainnumber), 1):
            c = str(longdump[j]).replace('[','').replace(']','')
            c = c.replace("'",'').replace(',','') + '\n'
            fwrite1.write(c)
        for j in range(0, len(shortchainnumber), 1):          
            f = str(shortdump[j]).replace('[','').replace(']','')
            f = f.replace("'",'').replace(',','') + '\n'
            fwrite2.write(f)
        fwrite1.close()
        fwrite2.close()
        
        #write timestep into a new file
        fwrite1 = open('long_time.txt', 'w')
        fwrite2 = open('short_time.txt', 'w')
        a = 'timestep' + '\n'
        d = 'timestep' + '\n'
        fwrite1.write(a)
        fwrite2.write(d)
        for j in range(0, len(timesteplong), 1):
            a = str(timesteplong[j]).replace('[', '').replace(']', '')
            a = a.replace("'", '').replace(',', '') + '\n'
            fwrite1.write(a)
        for j in range(0, len(timestepshort), 1):          
            d = str(timestepshort[j]).replace('[', '').replace(']', '')
            d = d.replace("'", '').replace(',', '') + '\n'
            fwrite2.write(d)
        fwrite1.close()
        fwrite2.close()
        
        #write chain number into a new file
        fwrite1 = open('long_chainnumber.txt', 'w')
        fwrite2 = open('short_chainnumber.txt', 'w')
        a = 'chainnumber' + '\n'
        d = 'chainnumber' + '\n'
        fwrite1.write(a)
        fwrite2.write(d)
        for j in range(0, len(longchainnumber), 1):
            a = str(longchainnumber[j]).replace('[', '').replace(']', '')
            a = a.replace("'", '').replace(',', '') + '\n'
            fwrite1.write(a)
        for j in range(0, len(shortchainnumber), 1):          
            d = str(shortchainnumber[j]).replace('[', '').replace(']', '')
            d = d.replace("'", '').replace(',', '') + '\n'
            fwrite2.write(d)
        fwrite1.close()
        fwrite2.close()
            
            
#2_pandas: particles in pore

        #load files
        dfl = pd.read_table('long_rearrange.txt', delim_whitespace=True)
        dfs = pd.read_table('short_rearrange.txt', delim_whitespace=True)
        dfl_t = pd.read_table('long_time.txt', delim_whitespace=True)
        dfs_t = pd.read_table('short_time.txt', delim_whitespace=True)
        dfl_c = pd.read_table('long_chainnumber.txt', delim_whitespace=True)
        dfs_c = pd.read_table('short_chainnumber.txt', delim_whitespace=True)
    
        #calculate real coordination
        xbox = 50 #the total length of simulation box
        ybox = 50 #the total length of simulation box
        zbox = 101 #the total length of simulation box
        dfl['realx'] = dfl.x + xbox*dfl.ix
        dfl['realy'] = dfl.y + ybox*dfl.iy
        dfl['realz'] = dfl.z + zbox*dfl.iz
        dfs['realx'] = dfs.x + xbox*dfs.ix
        dfs['realy'] = dfs.y + ybox*dfs.iy
        dfs['realz'] = dfs.z + zbox*dfs.iz
        
        #add timestep
        dfl['timestep'] = dfl_t['timestep']
        dfs['timestep'] = dfs_t['timestep']
        #add chainnumber
        dfl['chainnumber'] = dfl_c['chainnumber']
        dfs['chainnumber'] = dfs_c['chainnumber']
        
        #select particles in pore
        dfl_in_pore = dfl.loc[dfl['realz'] < 0]
        dfs_in_pore = dfs.loc[dfs['realz'] < 0]
            
        #write files
        dfl_in_pore.to_csv('long_particles_in_pore.txt', sep=' ', header=None, index=None)
        dfs_in_pore.to_csv('short_particles_in_pore.txt', sep=' ', header=None, index=None)


In [12]:
#3_whole_chain_in_pore

#load file
freadinital1 = open('long_particles_in_pore.txt', 'r')
longinitial = freadinital1.readlines()
freadinital2 = open('short_particles_in_pore.txt', 'r')
shortinitial = freadinital2.readlines()

# strip and split data
long = []
short = []
for j in longinitial:
    first = j.strip('/n')
    second = first.split()
    long.append(second)
for j in shortinitial:
    first = j.strip('/n')
    second = first.split()
    short.append(second)

#whether whole chain in pore
j = 1
lwhole = []
swhole = []
lchainnumber = int(long[0][12])
schainnumber = int(short[0][12])
ltime = int(long[0][11])
stime = int(short[0][11])
lmolecular = int(long[0][0])
smolecular = int(short[0][0])
longchainlength = 120 #chain length
shortchainlength = 40 #chain length
for k in range(1, len(long)-1,1):
    if int(long[k][12]) == int(lchainnumber) and int(long[k][11]) == int(ltime):
        if int(long[k][0]) != int(lmolecular):
            j = j + 1
            lchainnumber = int(long[k][12])
            ltime = int(long[k][11])
            lmolecular = int(long[k][0])
        if j == int(longchainlength):
            lwhole.append(long[k])
            j = 1
j = 1
for k in range(1, len(short)-1,1):
    if int(short[k][12]) == int(schainnumber) and int(short[k][11]) == int(stime):
        if int(short[k][0]) != int(smolecular):
            j = j + 1
            schainnumber = int(short[k][12])
            stime = int(short[k][11])
            smolecular = int(short[k][0])
        if j == int(shortchainlength):
            swhole.append(short[k])
            j = 1
        
#write into a new file
fwrite1 = open('long_whole_chain_in_pore.txt', 'w')
fwrite2 = open('short_whole_chain_in_pore.txt', 'w')
for j in range(0, len(lwhole), 1):
    s = str(lwhole[j]).replace('[','').replace(']','')
    s = s.replace("'",'').replace(',','') +'\n'
    fwrite1.write(s)
for j in range(0, len(swhole), 1):
    a = str(swhole[j]).replace('[','').replace(']','')
    a = a.replace("'",'').replace(',','') +'\n'
    fwrite2.write(a)
fwrite1.close()
fwrite2.close()
